# <center>Vakinha - Peticos 🐶😺</center>

Imports:

In [28]:
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from time import sleep
from selenium import webdriver
import psycopg2 as pg
import os
from dotenv import load_dotenv

#### Conectando com o Banco:

In [29]:
#Pegando a senha do banco:
load_dotenv()
password = os.getenv('PASSWORD')

In [30]:
#Conectando com o banco:
conn = pg.connect(
    dbname = "dbPeticos_2ano",
    user = "avnadmin",
    password = password,
    host = "db-peticos-cardosogih.k.aivencloud.com",
    port = 16207
)
cursor = conn.cursor()

Obtendo o link da vakinha:

In [31]:
#Armazendo o link da Vakinha em uma variável:
link_vakinha = "https://www.vakinha.com.br/vaquinha/cirurgia-da-mila-murilo-corsino?utm_internal_source=search_campaign"

#### Abrindo site da vakinha:

In [32]:
#Definindo para abrir a página Web em segundo plano:
chrome_options = Options()
chrome_options.add_argument("--headless")

#"Abrindo" o Chrome:
driver_vakinha = webdriver.Chrome(options=chrome_options)

#Abrindo o site da vakinha:
driver_vakinha.get(link_vakinha)

#### Pegando os valores: 

In [33]:
#Obtendo título da vakinha:
title = str(driver_vakinha.find_element(By.XPATH, '//*[@id="__next"]/div[4]/div[1]/div/div/h1').text)

#Obtendo a imagem da vakinha:
image_temp = driver_vakinha.find_element(By.XPATH, '//*[@id="__next"]/div[4]/div[2]/div[1]/div/a/div/picture/div/img')

#Obetando descrição da vakinha:
description = str(driver_vakinha.find_element(By.XPATH, '//*[@id="__next"]/div[4]/div[3]/div[2]/div/div[4]/p').text)

#Obtendo o valor arrecado na vakinha:
total_donated_temp = driver_vakinha.find_element(By.XPATH, '//*[@id="__next"]/div[4]/div[2]/div[2]/div/div/div/div[3]/div/span').text

#Obtendo a meta da vakinha:
goal_amount_temp = driver_vakinha.find_element(By.XPATH, '//*[@id="__next"]/div[4]/div[2]/div[2]/div/div/div/div[3]/div/div[3]/span').text

#Obtendo a quantidade de pessoas que já doaram:
supporters_amount_temp = driver_vakinha.find_element(By.XPATH, '//*[@id="__next"]/div[4]/div[2]/div[2]/div/div/div/div[3]/div/div[4]/span').text

#### Tratando os dados:

In [34]:
#-------------------------------------------------------------
#Tratando o campo imagem:
image = str(image_temp.get_attribute('src'))

#-------------------------------------------------------------
#Tratando o campo total_donated:
#Substituindo "R$" por vazio:
total_donated_temp = total_donated_temp.replace("R$ ", "")

#Substituindo o "." por vazio:
total_donated_temp = total_donated_temp.replace(".", "")

#Substituindo "," por ".", para poder transformar em número:
total_donated = float(total_donated_temp.replace(",", "."))

#-------------------------------------------------------------
#Tratando o campo goal_amount:
#Substituindo "R$" por vazio, para poder transformar em número:
goal_amount_temp = goal_amount_temp.replace("R$ ", "")

#Substituindo o "." por vazio:
goal_amount_temp = goal_amount_temp.replace(".", "")

#Substituindo "," por ".", para poder transformar em número:
goal_amount = float(goal_amount_temp.replace(",", "."))

#-------------------------------------------------------------
#Tratando o campo supporters_amount:
#Transformando o campo em int:
supporters_amount = int(supporters_amount_temp)

#-------------------------------------------------------------
#Calculando porcentagem arrecadada do total:
total_percentage = round(total_donated/goal_amount, 2) * 100

In [35]:
print(title)
print(total_donated)
print(goal_amount)
print(supporters_amount)
print(description)
print(total_percentage)
print(image)

id_pet = 2

Cirurgia da Mila
1235.0
2700.0
21
Olá, eu sou o Murilo, eu e minha mãe temos uma gatinha chamada Mila, ela tem 6 anos de idade e acabou caindo do quinto andar na quinta-feira do dia 19/09, desde então ela fez alguns exames no veterinário, onde o médico ortopedista informou que ela deslocou a Coluna da Pelvica e isso temos que realizar uma cirurgia de emergência onde A cirurgia tem intuito de estabilizar o quadril para manter a qualidade da marcha e manter o canal pélvico aberto para que às fezes possam fluir sem estreitamento, foi nos passado um valor de 2200 reais + 500 da anestesia, onde eu e minha mãe não temos como arcar com esse valor neste momento, pedimos a colaboração de todos para ajudar nossa gatinha voltar a andar e realizar a cirurgia.  
46.0
https://static.vakinha.com.br/uploads/vakinha/image/5100296/1727182399.681.png?ims=700x410


#### Inserindo dados no banco:

In [36]:
#Executando procedure para inserir os dados no banco:
cursor.execute("CALL insert_vakinha (%s, %s, %s, %s, %s, %s, %s, %s, %s)", (id_pet, title, link_vakinha, total_donated, goal_amount, supporters_amount, total_percentage, description, image))

#Comitando:
conn.commit()

#Desconectando:
cursor.close()
conn.close()